In [ ]:
from functools import partial
from pathlib import Path

import pandas as pd
import geopandas as gdp
from sepal_ui import sepalwidgets as sw
from sepal_ui import mapping as sm
import ipyvuetify as v

from utils import messages as ms
from utils import parameters as pm
from scripts import table as run

In [ ]:
#input and output as mutable objects 
class Table_io:
    def __init__(self):
        #input 
        self.json_table = None
        
        #output
        self.pts = None
        
tb_io = Table_io()

In [ ]:
# output to display messages 
test_output = sw.Alert()

# btn 
test_btn = sw.Btn(ms.TEST_BTN, icon='mdi-check')

# add some text
test_text = sw.Markdown(ms.TEST_TEXT)

#create a test downloader
id_ = 'file_widget'
title = "Download test data File"

test_tile = sw.Tile(
    id_,
    title,
    btn = test_btn,
    inputs=[test_text],
    output = test_output
)

In [ ]:
#output to display messages
file_output = sw.Alert().add_msg(ms.TABLE_INTRO)

#btn 
file_btn = sw.Btn(ms.TABLE_BTN, icon='mdi-check')

#create the inputs
file_select = sw.LoadTableField()
file_output.bind(file_select, tb_io, 'json_table')

#create a file selector 
id_ = 'file_widget'
title = "Select File"

file_tile = sw.Tile(
    id_,
    title,
    btn=file_btn, 
    inputs=[file_select],
    output=file_output
)


In [ ]:
#tiles to display the points on a map
tb_map = sm.SepalMap()
title = 'Display points'
tb_map_tile = sw.Tile(
    id_,
    title,
    inputs=[tb_map]
)

In [ ]:
#function to actualise the values of the select column tile 
def update_col(cols, values):
    for col in cols:
        col.items = values
        col.v_model=None

In [ ]:
import json

def load_file(widget, event, data, output, m):
    
    # toggle the loading button 
    widget.toggle_loading()
    
    # define variable 
    table = json.loads(tb_io.json_table)
    file = table['pathname']
    lat = table['lat_column']
    lng = table['lng_column']
    id_ = table['id_column']
    
    
    # check the variables 
    if not output.check_input(file, ms.NOT_A_FILE): return widget.toggle_loading()
    if not output.check_input(lat, ms.MISSING_INPUT): return widget.toggle_input()
    if not output.check_input(lng, ms.MISSING_INPUT): return widget.toggle_input()
    if not output.check_input(id_, ms.MISSING_INPUT): return widget.toggle_input()    
    
    # verify that they are all unique
    if len(set([lat, lng, id_])) != len([lat, lng, id_]): 
        output.add_msg(ms.REPEATED_INPUT, 'error')
        return widget.toggle_loading()
    
    try:
        # create the pts geodataframe
        df = pd.read_csv(file, sep=None, engine='python')
        df = df.filter(items=[lat, lng, id_])
        df = df.rename(columns={lat: 'lat', lng: 'lng', id_: 'id'})
        gdf = gdp.GeoDataFrame(df, geometry=gdp.points_from_xy(df.lng, df.lat), crs='EPSG:4326')
    
        # load the map
        run.setMap(gdf, m) 
    
        # set the dataframe in output 
        tb_io.pts = gdf
    
        output.add_msg(ms.VALID_COLUMNS, 'success')
        
    except Exception as e: 
        output.add_live_msg(str(e), 'error')
    
    # toggle the loading button 
    widget.toggle_loading()
    
    return 

file_btn.on_event('click', partial(
    load_file,
    output = file_output,
    m = tb_map
))

In [ ]:
def import_test_file(widget, event, data, output):

    widget.toggle_loading()

    # download the test dataset to the download folder 
    test_file = run.download_test_file(output)
    
    # add the file name to the file selector
    # need to update the fileinput component
    
    widget.toggle_loading()
    
    return 

test_btn.on_event('click', partial(
    import_test_file,
    output = test_output
))

In [ ]:
test_tile

In [ ]:
file_tile

In [ ]:
tb_map_tile